# Advent of Code

Solutions for Advent of Code 2023 puzzles.

In [1]:
import re
import itertools
import math

## Day 1

In [485]:
def day1_1():
    with open("day1.txt", "r") as f:
        data = f.readlines()
    digits = [re.findall("[0-9]", d) for d in data]
    result = sum([(int(d[0] + d[-1])) for d in digits])
    return result

In [486]:
day1_1() # 54877

54877

In [487]:
def day1_2():
    with open("day1.txt", "r") as f:
        data = f.readlines()

    words = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
    }
    special_cases = []
    # Special cases for combinations like 'eighthree'
    for a, b in itertools.permutations(list(words.keys()), 2):
        if a[-1] == b[0]:
            special_cases.append((a[:-1] + b, a + b))

    for text, replacement in special_cases:
        data = [d.replace(text, replacement) for d in data]

    digits = [
        re.findall("(one|two|three|four|five|six|seven|eight|nine|[0-9])", s)
        for s in data
    ]

    pure_digits = []
    for s in digits:
        values = [words.get(v, v) for v in s]
        pure_digits.append(values)

    result = sum([int(str(s[0]) + str(s[-1])) for s in pure_digits])

    return result

In [488]:
day1_2() # 54100

54100

## Day 2

In [476]:
def parser2(data):
    parsed_data = []
    for line in data:
        game = int(re.findall('Game ([0-9]+)', line)[0])
        subsets_raw = re.sub(r'Game ([0-9]+): ', '', line).split(";")
        subsets = []
        for subset in subsets_raw:
            extract_number = lambda s, color: int(re.findall(f'([0-9]+) {color}', s)[0]) if re.findall(f'([0-9]+) {color}', s) else 0
            red = extract_number(subset, 'red')
            blue = extract_number(subset, 'blue')
            green = extract_number(subset, 'green')

            subsets.append((red,blue,green))
        
        parsed_data.append((game, subsets))
    return parsed_data

In [489]:
def day2_1():
    with open("day2.txt", "r") as f:
        data = f.readlines()
    games = parser2(data)
    
    bag = (12, 14, 13) # red, blue, green
    possible_games, impossible_games = set(), set()
    for game in games:
        n, subsets = game[0], game[1]
        possible_games.add(n)
        for subset in subsets:
            if subset[0] > bag[0] or subset[1] > bag[1] or subset[2] > bag[2]:
                impossible_games.add(n)
                break
    possible_games = possible_games - impossible_games
    result = sum(possible_games)
    
    return result

In [490]:
day2_1() # 1931

1931

In [491]:
def day2_2():
    with open("day2.txt", "r") as f:
        data = f.readlines()
    games = parser2(data)
    
    bag = (12, 14, 13) # red, blue, green
    result = 0
    for game in games:
        reds = max([s[0] for s in game[1]])
        blues = max([s[1] for s in game[1]])
        greens = max([s[2] for s in game[1]])
        result += reds * blues *greens
    return result

In [493]:
day2_2() # 83105

83105

## Day 3

In [494]:
def parser3(filter_ch=None):
    with open("day3.txt", "r") as f:
        data = f.read().splitlines() 
    lines = data
    arr = [] # each value is a tuple (idx, value, parent_number)
    
    numbers = []
    symbols = [] # symbol's positions
    
    for i, line in enumerate(lines):
        line_vector = []
        for j, ch in enumerate(line):
            line_vector.append(ch)
            if ch != '.' and not ch.isdigit():
                if filter_ch and ch == filter_ch:
                    symbols.append((i,j))
                else:
                    symbols.append((i,j))
        arr.append(line_vector)
    
    return arr, symbols

In [495]:
def day3_1():
    
    arr, symbols = parser3()
    
    part_numbers = []
    lx, ly = len(arr), len(arr[0])
    visited_cells = []
    
    def is_cell_allowed(row, col, lx=lx, ly=ly):
        return row >= 0 and row < lx and col >= 0 and col < ly
    
    for symbol in symbols:
        i, j = symbol[0], symbol[1]
        # navigate around the symbol
        nodes = [(-1,1), (0,1), (1,1), (1,0), (1, -1), (0, -1), (-1,-1), (-1,0)]
        for (di, dj) in nodes:
            if is_cell_allowed(row=i + di, col=j + dj):
                cell = arr[i + di][j + dj]
                if (i + di, j + dj) not in visited_cells and cell.isdigit():
                    part_number = cell
                    # look to the left
                    row, col = i + di, j + dj - 1
                    if is_cell_allowed(row, col):
                        previous = arr[row][col]
                        visited_cells.append((row,col))
                        while previous.isdigit():
                            part_number = previous + part_number
                            col -= 1
                            if is_cell_allowed(row, col):
                                previous = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break
                    # look to the right
                    row, col = i + di, j + dj + 1
                    if is_cell_allowed(row, col):
                        next_cell = arr[row][col]
                        visited_cells.append((row,col))
                        while next_cell.isdigit():
                            part_number = part_number + next_cell
                            col += 1
                            if is_cell_allowed(row, col):
                                next_cell = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break

                    part_numbers.append(part_number)
                    
    part_numbers = [int(v) for v in part_numbers]
    return sum(part_numbers)

In [497]:
day3_1() # 525119

525119

In [498]:
def day3_2():
    arr, symbols = parser3(filter_ch="*")  
    
    all_part_numbers = []
    lx, ly = len(arr), len(arr[0])
    visited_cells = []
    
    def is_cell_allowed(row, col, lx=lx, ly=ly):
        return row >= 0 and row < lx and col >= 0 and col < ly
    
    
    for symbol in symbols: # only gears
        i, j = symbol[0], symbol[1]
        part_numbers = []
        
        # navigate around the symbol
        nodes = [(-1,1), (0,1), (1,1), (1,0), (1, -1), (0, -1), (-1,-1), (-1,0)]
        for (di, dj) in nodes:
            if is_cell_allowed(row=i + di, col=j + dj):
                cell = arr[i + di][j + dj]
                if (i + di, j + dj) not in visited_cells and cell.isdigit():
                    part_number = cell
                    # look to the left
                    row, col = i + di, j + dj - 1
                    if is_cell_allowed(row, col):
                        previous = arr[row][col]
                        visited_cells.append((row,col))
                        while previous.isdigit():
                            part_number = previous + part_number
                            col -= 1
                            if is_cell_allowed(row, col):
                                previous = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break
                                
                    # look to the right
                    row, col = i + di, j + dj + 1
                    if is_cell_allowed(row, col):
                        next_cell = arr[row][col]
                        visited_cells.append((row,col))
                        while next_cell.isdigit():
                            part_number = part_number + next_cell
                            col += 1
                            if is_cell_allowed(row, col):
                                next_cell = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break

                    part_numbers.append(part_number)
        
        part_numbers = [int(v) for v in part_numbers]
        all_part_numbers.append(part_numbers)
                    
    # after all symbols, check for gears
    gear_ratios = []
    for symbol_numbers in all_part_numbers:
        if symbol_numbers and len(symbol_numbers) == 2:
            gear_ratios.append(math.prod(symbol_numbers))
    
    return sum(gear_ratios)

In [500]:
day3_2() # 76504829

76504829

## Day 4

In [501]:
def day4_1():
    with open("day4.txt", "r") as f:
        data = f.read().splitlines()
    points = 0
    for d in data:
        winning = set(re.findall('([0-9\s]+) \|', d)[0].split())
        mine = set(re.findall('\| ([0-9\s]+)', d)[0].split())
        if mine.intersection(winning):
            points += 2 ** (len(mine.intersection(winning)) - 1)
            
    return points

In [502]:
day4_1() # 21558

21558

In [503]:
def day4_2():
    with open("day4.txt", "r") as f:
        cards = f.read().splitlines()
    
    deck = {} # dict with card number and number of copies
    cards = cards
    
    for card in cards:
        idx = int(re.findall('Card\s+([0-9]+):', card)[0])
        if idx not in deck:
            deck[idx] = 1
    
    for card in cards:
        idx = int(re.findall('Card\s+([0-9]+):', card)[0])
        copies = deck.get(idx)
        winning = set(re.findall('([0-9\s]+) \|', card)[0].split())
        mine = set(re.findall('\| ([0-9\s]+)', card)[0].split())
            
        matchs = len(mine.intersection(winning))
        
        # get copies
        for i in range(idx + 1, idx + matchs + 1, 1):
            if i not in deck:
                deck[i] = 1
            else:
                deck[i] += copies # each copy gives new copies

    return sum(deck.values())

In [504]:
day4_2() # 10425665

10425665

## Day 5

In [506]:
def parser5(sample=True, part2=False):
    sample = ""if not sample else "samples/"
    with open(f"{sample}day5.txt", "r") as f:
        data = f.read()
        data = re.sub('\n+', ' ', data)
    
    real_seeds = []
    if part2:
        seeds = re.findall('seeds: ([0-9\s]+)', data)[0].split()
        seeds = [int(v) for v in seeds]
        for i in range(0, len(seeds), 2):
            real_seeds.append((seeds[i], seeds[i+1]))
        seeds = real_seeds
    else:
        seeds = re.findall('seeds: ([0-9\s]+)', data)[0].split()
        seeds = [int(v) for v in seeds]
    
    maps = re.findall("([\w]+)-to-([\w]+) map: ([0-9\s]+)", data)
    maps = [(dest, src, re.split('([0-9]+\s[0-9]+\s[0-9]+)', v) ) for (dest, src, v) in maps]
    maps = [(dest, src, [v for v in v if not re.match('\s+', v) and v])  for (dest, src, v) in maps]
    maps = [(dest, src, [tuple([int(v) for v in v.split()]) for v in v])  for (dest, src, v) in maps]
    
    return seeds, maps

In [509]:
def day5_1(sample=False):
    seeds, maps = parser5(sample, part2=False)
    locs = []

    def f(x, mapper):
        for (dest, src, rng) in mapper[2]:
            if x >= src and x <= src + rng - 1:
                x = dest + (x - src)
                break
        return x
    
    for seed in seeds:
        x = seed
        for mapper in maps: 
            x = f(x, mapper)
        locs.append(x)
        
    return min(locs)

In [510]:
%%time
day5_1(sample=False) # 650599855

CPU times: user 1.56 ms, sys: 837 µs, total: 2.39 ms
Wall time: 2.16 ms


650599855

In [518]:
def day5_2(sample=False):
    seeds, maps = parser5(sample, part2=True)
    locs = []
        
    def f_inv(y, mapper):
        x = y
        for (dest, src, rng) in mapper[2]:
            if y >= dest and y <= dest + rng - 1:
                x = src + (y - dest)
                break
        return x

    inv_maps = maps[::-1]
    locations_map = inv_maps[0][2]
    
    for location in range(0, max([l[0] for l in locations_map])):
        x = location
        for mapper in inv_maps:
            x = f_inv(x, mapper)
        for (seed, rng) in seeds:
            if x >= seed and x <= seed + rng -1:
                return location

In [519]:
%%time
day5_2(sample=False) # 1240035

CPU times: user 11.4 s, sys: 36.9 ms, total: 11.5 s
Wall time: 11.6 s


1240035

## Day 6

In [513]:
def parser6(sample=True, part2=False):
    sample = ""if not sample else "samples/"
    with open(f"{sample}day6.txt", "r") as f:
         data = f.readlines()
    
    if part2:
        data[0] = re.sub('\s+', '', data[0])
        data[1] = re.sub('\s+', '', data[1])
    times = [int(v) for v in re.findall('([0-9]+)', data[0])]
    distances = [int(v) for v in re.findall('([0-9]+)', data[1])]
    
    return times, distances

In [516]:
import math 

def day6_1(sample=True, part2=False):
    times, distances = parser6(sample, part2)
    
    possibilities = []
    for (t, d) in zip(times, distances):
        delta = (t ** 2 - 4 * d) ** 0.5 / 2
        x1, x2 = max(0, t/2 - delta), max(0, t/2 + delta)
        l1 = math.ceil(x1) if int(x1) != x1 else x1 + 1
        l2 = math.floor(x2) if int(x2) != x2 else x2 -1
        possibilities.append(l2 - l1 + 1)
        
    
    return math.prod(possibilities)

In [520]:
day6_1(sample=False) # 2374848

2374848

In [522]:
day6_1(sample=False, part2=True) # 39132886

39132886

## Day 7

In [534]:
def hand_points(hand: str, deck, names, check_jokers=False) -> str:    
    fullhand = {d: 0 for d in deck}
    for card in hand:
        fullhand[card] += 1

    jokers = 0
    if check_jokers:
        jokers = len([card for card in hand if card == 'J'])

    card_group = lambda fullhand, value:  [v for v in fullhand.values() if v == value]

    if len(set(list(hand))) == 1: # five of a kind
        return 7
    elif card_group(fullhand, 4): # four of a kind
        bonus = 1 if jokers > 0 else 0
        return 6 + bonus
    elif card_group(fullhand, 3) and card_group(fullhand, 2): # full house
        bonus = 2 if jokers > 0 else 0
        return 5 + bonus
    elif card_group(fullhand, 3) and not card_group(fullhand, 2): # three of a kind
        bonus = 2 if jokers > 0 else 0
        return 4 + bonus
    elif len(card_group(fullhand, 2)) == 2: # two pair
        bonus = jokers + 1 if jokers > 0 else 0
        assert jokers in (0,1,2)
        return 3 + bonus
    elif len(card_group(fullhand, 2)) == 1: # one pair
        bonus = 2 if jokers > 0 else 0
        return 2 + bonus
    elif len(set(list(hand))) == len(list(hand)): # high card
        bonus = 1 if jokers > 0 else 0
        return 1 + bonus

def day7(sample=True, part2=False):
    sample = "" if not sample else "samples/"
    with open(f"{sample}day7.txt", "r") as f:
         data = f.readlines()
    data = [d.split() for d in data]
    bids = dict(data)
    hands = list(bids.keys())
    assert len(hands) == len(bids)
    if part2:
        deck = list('AKQT98765432J')
    else:
        deck = list('AKQJT98765432')
        
    names = {
            7: "Five of a Kind",
            6: "Four of a Kind",
            5: "Full House",
            4: "Three of a Kind",
            3: "Two Pair",
            2: "One Pair",
            1: "High Card"
        }
    
    import functools
    def compare_hands(a, b):
        a_pts = hand_points(a, deck, names, check_jokers=part2)
        b_pts = hand_points(b, deck, names, check_jokers=part2)
        if a_pts > b_pts:
            return 1
        elif a_pts < b_pts:
            return -1
        else:
            for ch1, ch2 in zip(a, b):
                if deck.index(ch1) != deck.index(ch2):
                    if deck.index(ch1) < deck.index(ch2):
                        return 1
                    else:
                        return -1
        
    sorted_hands = sorted(hands, key=functools.cmp_to_key(compare_hands))
    total_points = 0
    for hand in sorted_hands:
        total_points += (sorted_hands.index(hand) + 1) * int(bids.get(hand))
        
    if sample: print(bids)
            
    return total_points

In [535]:
day7(sample=False) # 250946742

250946742

In [536]:
day7(sample=False, part2=True) # 251824095

251824095

## Day 8

In [538]:
def parser8(sample='a', part2=False):
    path = "day8.txt" if not sample else f"samples/day8{sample}.txt"
    with open(path, "r") as f:
         data = f.readlines()
    instructions = re.findall('\w+', data[0])
    nodes = {}
    for d in data[2:]:
        node, maps = d.split('=')
        nodes[re.findall('(\w+)', node)[0]] = re.findall('(\w{3})', maps)
            
    return instructions[0], nodes

In [548]:
def day8(sample='a', part2=False):
    instructions, nodes_map = parser8(sample, part2)
    n_i = len(instructions)
    
    current_nodes = ['AAA'] if not part2 else [node for node in nodes_map.keys() if node[-1] == 'A']
    
    n_nodes = len(current_nodes)
    finishes = []
    step = 0
    
    while len(finishes) != n_nodes:
        step += 1
        r = step % n_i
        idx = n_i - 1 if r == 0 else r - 1
        instruction = instructions[idx]
        instruction = 0 if instruction == 'L' else 1
        
        next_nodes = []
        for node in current_nodes:
            next_node = nodes_map[node][instruction]
            if next_node[-1] == 'Z':
                finishes.append(step)
            else:
                next_nodes.append(next_node)
    
        current_nodes = next_nodes
    
    return math.lcm(*finishes)

In [549]:
day8(sample=False) # 19667

19667

In [551]:
day8(sample=False, part2=True) # 19185263738117

19185263738117

## Day 9

In [552]:
def parser9(sample=True, part2=False):
    sample = ""if not sample else "samples/"
    with open(f"{sample}day9.txt", "r") as f:
        data = f.readlines()
    data = [re.sub('\n', '', d).split() for d in data]
    data = [[int(v) for v in d] for d in data]

    return data

In [557]:
def day9(sample=True, part2=False):
    data = parser9(sample, part2)
    forecasts = []
    for series in data:
        diffs = [series]
        get_next_diff = lambda diff: [diff[i] - diff[i - 1] for i in range(1, len(diff))]
        diff = series
        while diff[-1] != 0:
            next_diff = get_next_diff(diff)
            diffs.append(next_diff)
            diff = next_diff
        
        if not part2:
            forecasts.append(sum([l[-1] for l in diffs]))
        else:
            forecast = sum([l[0] * (-1) ** i for i, l in enumerate(diffs)])            
            forecasts.append(forecast)
    
    return sum(forecasts)

In [558]:
day9(sample=False, part2=False) # 1995001648

1995001648

In [560]:
day9(sample=False, part2=True) # 988

988